In [36]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [37]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [39]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.pkl


In [40]:
len(testset[testset['topics'].str.len() > 0])

69817

In [41]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t1+1.1+1.2+1.3+1.4


In [42]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.1.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.2.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.3.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t1.4.pkl


In [43]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

69817

In [44]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [45]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [46]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [47]:
if len(merge_list)>0:
    testset = testset_merge

In [48]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [49]:
orig_testset = testset

In [50]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 69817
topics came as list ... consider joining it before


In [51]:
runtime_vectorizer

0.7370486259460449

In [16]:
word_doc_matrix

<69817x11764 sparse matrix of type '<class 'numpy.float64'>'
	with 562674 stored elements in Compressed Sparse Row format>

In [17]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [18]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 34362 edges.
It has only fully disconnected 761 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 12min 45s, sys: 12.4 s, total: 12min 58s
Wall time: 3min 9s


In [19]:
print(runtime_umap)

189.77288174629211


In [20]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 2min 34s, sys: 585 ms, total: 2min 34s
Wall time: 2min 16s


In [21]:
print(runtime_umap_transform)

136.48599362373352


In [22]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/umap_embedding_t1+1.1.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/umap_clusterable_embedding_t1+1.1.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [23]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.74225,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
4,hdb,t1+1.1,1,ss80cs80,80,80,1.381863,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
4,hdb,t1+1.1,1,ss80cs80,80,80,1.381863,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
5,hdb,t1+1.1,1,ss110cs80,80,110,1.547739,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
4,hdb,t1+1.1,1,ss80cs80,80,80,1.381863,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
5,hdb,t1+1.1,1,ss110cs80,80,110,1.547739,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
6,hdb,t1+1.1,1,ss50cs110,110,50,1.235591,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
4,hdb,t1+1.1,1,ss80cs80,80,80,1.381863,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
5,hdb,t1+1.1,1,ss110cs80,80,110,1.547739,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
6,hdb,t1+1.1,1,ss50cs110,110,50,1.235591,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
7,hdb,t1+1.1,1,ss80cs110,110,80,1.421608,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t1+1.1,1,ss50cs50,50,50,1.742250,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
1,hdb,t1+1.1,1,ss80cs50,50,80,1.314299,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
2,hdb,t1+1.1,1,ss110cs50,50,110,1.577260,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
3,hdb,t1+1.1,1,ss50cs80,80,50,1.205789,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
4,hdb,t1+1.1,1,ss80cs80,80,80,1.381863,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
5,hdb,t1+1.1,1,ss110cs80,80,110,1.547739,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
6,hdb,t1+1.1,1,ss50cs110,110,50,1.235591,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
7,hdb,t1+1.1,1,ss80cs110,110,80,1.421608,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."
8,hdb,t1+1.1,1,ss110cs110,110,110,1.460693,"min_df: 4, max_df: 1.0, stopwords: True, exten...","hellinger, rt_umap_tf: 136.48599362373352, rt_..."


In [24]:
hdb_labels_results

,name,ss50cs50,ss80cs50,ss110cs50,ss50cs80,ss80cs80,ss110cs80,ss50cs110,ss80cs110,ss110cs110
0,thingsym/multi-device-switcher,11,11,9,9,11,9,8,9,9
1,redlink-gmbh/solr-compound-word-filter,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,xswlme/Delivery-Availability-Checker,39,25,19,35,25,19,30,21,19
3,eastsss/ErrorDispatching,52,49,32,43,45,29,34,36,29
4,ahmadhuzaifa/skinskan,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
69812,waqasbhatti/astrobase,50,40,-1,41,36,-1,-1,-1,-1
69813,hpcc-systems/vscode-ecl,-1,-1,-1,-1,-1,-1,-1,-1,-1
69814,MarcoPon/BlockHashLoc,94,89,77,76,80,69,60,68,64
69815,bhagatabhijeet/bhagatabhijeet,115,-1,-1,96,-1,60,74,67,57


In [25]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/hdb_labels_results_t1+1.1.pkl


In [26]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [27]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [28]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1,1,10,761,10,"min_df: 4, max_df: 1.0, stopwords: True, exten...",0.521003,"(hellinger, rt_umap_tf: 136.48599362373352, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t1+1.1,1,10,761,10,"min_df: 4, max_df: 1.0, stopwords: True, exten...",0.521003,"(hellinger, rt_umap_tf: 136.48599362373352, rt..."
1,kmeans-umapt,t1+1.1,1,13,761,13,"min_df: 4, max_df: 1.0, stopwords: True, exten...",0.486910,"(hellinger, rt_umap_tf: 136.48599362373352, rt..."


In [29]:
kmeans_labels_results

,name,10,13
0,thingsym/multi-device-switcher,6,8
1,redlink-gmbh/solr-compound-word-filter,8,0
2,xswlme/Delivery-Availability-Checker,6,3
3,eastsss/ErrorDispatching,6,8
4,ahmadhuzaifa/skinskan,5,10
...,...,...,...
69812,waqasbhatti/astrobase,0,9
69813,hpcc-systems/vscode-ecl,2,3
69814,MarcoPon/BlockHashLoc,4,0
69815,bhagatabhijeet/bhagatabhijeet,8,6


In [30]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans-umap_labels_results_t1+1.1.pkl


In [31]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t1+1.1,1,10,10,"min_df: 4, max_df: 1.0, stopwords: True, exten...",1.72243


,type,testset,version,size,name,tdf,runtime
0,kmeans,t1+1.1,1,10,10,"min_df: 4, max_df: 1.0, stopwords: True, exten...",1.722430
1,kmeans,t1+1.1,1,13,13,"min_df: 4, max_df: 1.0, stopwords: True, exten...",2.254365


In [32]:
kmeans_labels_results

,name,10,13
0,thingsym/multi-device-switcher,6,2
1,redlink-gmbh/solr-compound-word-filter,6,2
2,xswlme/Delivery-Availability-Checker,6,2
3,eastsss/ErrorDispatching,6,2
4,ahmadhuzaifa/skinskan,7,5
...,...,...,...
69812,waqasbhatti/astrobase,6,2
69813,hpcc-systems/vscode-ecl,6,2
69814,MarcoPon/BlockHashLoc,6,2
69815,bhagatabhijeet/bhagatabhijeet,6,2


In [33]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/test/kmeans_labels_results_t1+1.1.pkl
